In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Chargement des données
df_path = '../../data/processed/df_avec_categorie_part3_traduit.csv'
df = pd.read_csv(df_path)

# création de la colonne 'text' à partir de 'trad' ou 'truncated'
df['text'] = df.apply(lambda row: row['trad'] if not pd.isna(row['trad']) else row['truncated'], axis=1)

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['category_name']

# Split train/test stratifié pour conserver les proportions des classes
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#model = SVC(gamma=0.01, kernel='linear', class_weight= 'balanced')
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10], 'class_weight': ['balanced']}

svc_model = svm.SVC()
gridsearch = GridSearchCV(svc_model, parameters)
gridsearch.fit(X,y)

print("les meilleurs paramètres sont", gridsearch.best_params_)
print("le meilleur score obtenu est ", gridsearch.best_score_)

#les meilleurs paramètres sont {'C': 10, 'class_weight': 'balanced', 'kernel': 'linear'}
#le meilleur score obtenu est  0.7450774234905061


les meilleurs paramètres sont {'C': 10, 'class_weight': 'balanced', 'kernel': 'linear'}
le meilleur score obtenu est  0.7450774234905061


In [ ]:
from skopt import BayesSearchCV
parameters = {'kernel' : ['linear', 'rbf'], 'C' : (0.1, 10.0, 'log-uniform'), class_weight: 'balanced'}

svc_model = svm.SVC()
bayes_search = BayesSearchCV(svc_model, parameters,  n_iter = 20)
bayes_search.fit(X,y)

print("les meilleurs paramètres sont", bayes_search.best_params_)
print("le meilleur score obtenu est ", bayes_search.best_score_)